**In this guide we will demonstrate how to use Nano AutoML to auto tune an machine learning model and handle the whole model development process easily.**

# Introduction
Nano provides hpo as an solution for fast and high-performance hyperparameter tuning. Users can import hano hpo following the standard tensorflow API without changing their own code. By setting the search space and running model.search(), the model will contain the best hyperparameters.

Nano hpo supports sequential, fuctional and custom keras API.


# Step0: Prepare Environment
We recommend using conda to prepare the environment.
conda create -n my_env python=3.7

conda activate my_env

pip install --pre --upgrade bigdl

# Step1: Init Nano AutoML
Let's start out by importing Nano AutoML and enable its tensorflow API globally by automl.hpo_config.enable_hpo_tf().

After importing, we will load MNIST as our data.

In [1]:
import bigdl.nano.automl as automl
automl.hpo_config.enable_hpo_tf()

from bigdl.nano.tf.keras.layers import Dense
from bigdl.nano.tf.keras import Input
from bigdl.nano.automl.tf.keras import Model
import bigdl.nano.automl.hpo.space as space


import tensorflow as tf
from tensorflow import keras

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

../../src/bigdl/nano/tf/__init__.py:24: UserWarning: NANO_TF_INTER_OP not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("NANO_TF_INTER_OP not found the in os.environ, "
../../src/bigdl/nano/tf/__init__.py:30: UserWarning: OMP_NUM_THREADS not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("OMP_NUM_THREADS not found the in os.environ, "


# Step2: Set search space
We now create our model function. 

Whenever you want to search a hyperparameter, you can simply replace it with a space and set your own searching space 


In [2]:
inputs = Input(shape=(784,))
x = Dense(units=space.Categorical(8,16,prefix='dense_1'), activation="relu")(inputs)
x = Dense(units=space.Categorical(32,64,prefix='dense_2'), activation="relu")(x)
outputs = Dense(units=10)(x)

model = Model(inputs=inputs, outputs=outputs, name="mnist_model")

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

2022-05-19 16:55:19.758434: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-19 16:55:19.759291: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


# Step3: Run hyperparameter search
Apply hyperparameter searching by calling model.search(), and best hyperparameters will be saved in the model.

You can specify the number of trial, each trial will use a set of hyperparameters automatiacally suggusted by nano.hpo. 

You can set your target metric and whehter you want to maximize you minimize it.

In [3]:
%%time
model.search(
    n_trials=5,
    target_metric='val_accuracy',
    direction="maximize",
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=24,
    validation_split=0.2,
    verbose=False,
)

[I 2022-05-19 16:55:19,898] A new study created in memory with name: no-name-a5486f4b-d4ee-4821-8327-bbab12f6f57a


Starting a new tuning
Epoch 1/24
375/375 - 2s - loss: 0.8326 - accuracy: 0.7587 - val_loss: 0.3935 - val_accuracy: 0.8863 - 2s/epoch - 4ms/step
Epoch 2/24
375/375 - 1s - loss: 0.3641 - accuracy: 0.8938 - val_loss: 0.3080 - val_accuracy: 0.9105 - 1s/epoch - 4ms/step
Epoch 3/24
375/375 - 1s - loss: 0.3070 - accuracy: 0.9110 - val_loss: 0.2860 - val_accuracy: 0.9158 - 890ms/epoch - 2ms/step
Epoch 4/24
375/375 - 1s - loss: 0.2817 - accuracy: 0.9192 - val_loss: 0.2582 - val_accuracy: 0.9239 - 860ms/epoch - 2ms/step
Epoch 5/24
375/375 - 1s - loss: 0.2640 - accuracy: 0.9241 - val_loss: 0.2533 - val_accuracy: 0.9267 - 873ms/epoch - 2ms/step
Epoch 6/24
375/375 - 1s - loss: 0.2523 - accuracy: 0.9283 - val_loss: 0.2438 - val_accuracy: 0.9312 - 825ms/epoch - 2ms/step
Epoch 7/24
375/375 - 1s - loss: 0.2426 - accuracy: 0.9303 - val_loss: 0.2270 - val_accuracy: 0.9342 - 821ms/epoch - 2ms/step
Epoch 8/24
375/375 - 1s - loss: 0.2351 - accuracy: 0.9319 - val_loss: 0.2241 - val_accuracy: 0.9358 - 892ms/e

[I 2022-05-19 16:55:44,080] Trial 0 finished with value: 0.9430000185966492 and parameters: {'units▁choice': 0}. Best is trial 0 with value: 0.9430000185966492.


Epoch 1/24
375/375 - 2s - loss: 0.5912 - accuracy: 0.8408 - val_loss: 0.2848 - val_accuracy: 0.9187 - 2s/epoch - 5ms/step
Epoch 2/24
375/375 - 1s - loss: 0.2761 - accuracy: 0.9193 - val_loss: 0.2459 - val_accuracy: 0.9285 - 1s/epoch - 3ms/step
Epoch 3/24
375/375 - 1s - loss: 0.2329 - accuracy: 0.9319 - val_loss: 0.2117 - val_accuracy: 0.9387 - 1s/epoch - 3ms/step
Epoch 4/24
375/375 - 1s - loss: 0.2054 - accuracy: 0.9384 - val_loss: 0.1972 - val_accuracy: 0.9428 - 1s/epoch - 3ms/step
Epoch 5/24
375/375 - 1s - loss: 0.1861 - accuracy: 0.9449 - val_loss: 0.1817 - val_accuracy: 0.9466 - 1s/epoch - 3ms/step
Epoch 6/24
375/375 - 1s - loss: 0.1703 - accuracy: 0.9494 - val_loss: 0.1803 - val_accuracy: 0.9465 - 1s/epoch - 3ms/step
Epoch 7/24
375/375 - 1s - loss: 0.1594 - accuracy: 0.9521 - val_loss: 0.1654 - val_accuracy: 0.9526 - 1s/epoch - 3ms/step
Epoch 8/24
375/375 - 1s - loss: 0.1488 - accuracy: 0.9550 - val_loss: 0.1676 - val_accuracy: 0.9491 - 1s/epoch - 3ms/step
Epoch 9/24
375/375 - 1s 

[I 2022-05-19 16:56:11,459] Trial 1 finished with value: 0.9597499966621399 and parameters: {'units▁choice': 1}. Best is trial 1 with value: 0.9597499966621399.


...

Epoch 1/24
375/375 - 3s - loss: 0.5797 - accuracy: 0.8454 - val_loss: 0.2912 - val_accuracy: 0.9173 - 3s/epoch - 9ms/step
Epoch 2/24
375/375 - 1s - loss: 0.2844 - accuracy: 0.9172 - val_loss: 0.2598 - val_accuracy: 0.9227 - 975ms/epoch - 3ms/step
Epoch 3/24
375/375 - 1s - loss: 0.2368 - accuracy: 0.9317 - val_loss: 0.2124 - val_accuracy: 0.9380 - 906ms/epoch - 2ms/step
Epoch 4/24
375/375 - 1s - loss: 0.2075 - accuracy: 0.9397 - val_loss: 0.2025 - val_accuracy: 0.9398 - 1s/epoch - 3ms/step
Epoch 5/24
375/375 - 2s - loss: 0.1873 - accuracy: 0.9445 - val_loss: 0.1874 - val_accuracy: 0.9449 - 2s/epoch - 5ms/step
Epoch 6/24
375/375 - 1s - loss: 0.1727 - accuracy: 0.9495 - val_loss: 0.1770 - val_accuracy: 0.9494 - 1s/epoch - 4ms/step
Epoch 7/24
375/375 - 1s - loss: 0.1605 - accuracy: 0.9521 - val_loss: 0.1651 - val_accuracy: 0.9523 - 1s/epoch - 4ms/step
Epoch 8/24
375/375 - 1s - loss: 0.1498 - accuracy: 0.9555 - val_loss: 0.1592 - val_accuracy: 0.9550 - 888ms/epoch - 2ms/step
Epoch 9/24
375/

[I 2022-05-19 16:57:14,031] Trial 3 finished with value: 0.9605000019073486 and parameters: {'units▁choice': 1}. Best is trial 2 with value: 0.9610833525657654.


Epoch 1/24
375/375 - 2s - loss: 0.7677 - accuracy: 0.7797 - val_loss: 0.3537 - val_accuracy: 0.8987 - 2s/epoch - 4ms/step
Epoch 2/24
375/375 - 1s - loss: 0.3525 - accuracy: 0.8978 - val_loss: 0.3075 - val_accuracy: 0.9078 - 1s/epoch - 4ms/step
Epoch 3/24
375/375 - 1s - loss: 0.3090 - accuracy: 0.9094 - val_loss: 0.2791 - val_accuracy: 0.9162 - 1s/epoch - 3ms/step
Epoch 4/24
375/375 - 1s - loss: 0.2845 - accuracy: 0.9172 - val_loss: 0.2644 - val_accuracy: 0.9231 - 1s/epoch - 3ms/step
Epoch 5/24
375/375 - 1s - loss: 0.2689 - accuracy: 0.9217 - val_loss: 0.2580 - val_accuracy: 0.9250 - 1s/epoch - 3ms/step
Epoch 6/24
375/375 - 1s - loss: 0.2580 - accuracy: 0.9254 - val_loss: 0.2520 - val_accuracy: 0.9273 - 1s/epoch - 3ms/step
Epoch 7/24
375/375 - 1s - loss: 0.2493 - accuracy: 0.9276 - val_loss: 0.2388 - val_accuracy: 0.9315 - 1s/epoch - 3ms/step
Epoch 8/24
375/375 - 2s - loss: 0.2421 - accuracy: 0.9294 - val_loss: 0.2418 - val_accuracy: 0.9311 - 2s/epoch - 4ms/step
Epoch 9/24
375/375 - 1s 

[I 2022-05-19 16:57:39,883] Trial 4 finished with value: 0.9392499923706055 and parameters: {'units▁choice': 0}. Best is trial 2 with value: 0.9610833525657654.


CPU times: user 8min 15s, sys: 9min 57s, total: 18min 12s
Wall time: 2min 19s


# Step 4: fit with the best hyperparameters
The best hyperparameters have been saved in the model. You can call model.fit() to train a model with the best hyperparameters.

In [4]:
history = model.fit(x_train, y_train,
                    batch_size=128, epochs=24, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/24
375/375 [==============================] - 2s 4ms/step - loss: 0.5704 - accuracy: 0.8392 - val_loss: 0.2954 - val_accuracy: 0.9155
Epoch 2/24
375/375 [==============================] - 2s 5ms/step - loss: 0.2796 - accuracy: 0.9193 - val_loss: 0.2353 - val_accuracy: 0.9327
Epoch 3/24
375/375 [==============================] - 3s 9ms/step - loss: 0.2315 - accuracy: 0.9326 - val_loss: 0.2072 - val_accuracy: 0.9401
Epoch 4/24
375/375 [==============================] - 4s 11ms/step - loss: 0.2033 - accuracy: 0.9403 - val_loss: 0.1943 - val_accuracy: 0.9448
Epoch 5/24
375/375 [==============================] - 3s 8ms/step - loss: 0.1843 - accuracy: 0.9461 - val_loss: 0.1784 - val_accuracy: 0.9476
Epoch 6/24
375/375 [==============================] - 2s 4ms/step - loss: 0.1699 - accuracy: 0.9499 - val_loss: 0.1690 - val_accuracy: 0.9516
Epoch 7/24
375/375 [==============================] - 2s 4ms/step - loss: 0.1581 - accuracy: 0.9529 - val_loss: 0.1647 - val_accuracy: 0.9520
Epoch

Check out the summary of the model. The model has already been built with the best hyperparameters found by nano hpo.

In [5]:
print(model.summary())

Model: "mnist_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_3 (Dense)             (None, 16)                12560     
                                                                 
 dense_4 (Dense)             (None, 64)                1088      
                                                                 
 dense_5 (Dense)             (None, 10)                650       
                                                                 
Total params: 14,298
Trainable params: 14,298
Non-trainable params: 0
_________________________________________________________________
None
